In [4]:
import sys
sys.path.append("..")

import numpy as np
from propagation_simulator.environment import environment
from classes.nfpo import NFPO
from matplotlib import pyplot as plt

In [5]:
antennas_populations = np.arange(3,8,1)
antennas_spaces = np.arange(0.1,0.26,0.01)
k = -2
frequency = 920.4e6
c = 2.998e8
sample_count = 100
wave_length = c/frequency
x_list = np.arange(0,3,1)
y=2

In [ ]:
solve_time_matrix = np.zeros(shape=(antennas_populations.shape[0],antennas_spaces.shape[0]))
error_matrix = np.zeros(shape=(antennas_populations.shape[0],antennas_spaces.shape[0]))
for i, antennas_population in enumerate(antennas_populations):
    for j, antennas_space in enumerate(antennas_spaces):
        env = environment(
            antennas_space=antennas_space,
            antennas_population=antennas_population,
            frequency=frequency,
            tag=[0,2],
            k=k,
            sample_count=sample_count,
        )
        _, phases, _ = env.get_phases()
        nfpo = NFPO(
            sample_count=100,
            phases=phases,
            antennas_population=antennas_population,
            antennas_space=antennas_space,
            wave_length=wave_length,
            k=k,
            tag_location=[0,2],
            initial_value=[0,1.5],
            x_err_range=2,
            y_err_range=1.5,
            is_noise_included=True,
        )
        result = nfpo.solve(tee=False, time_limit=60)
        print(f"antennas population = {antennas_population}, antennas space = {antennas_space}")
        print(f"    Solve time = {result["solving_time"]}, Error = {result["values"]["error"]}")
        solve_time_matrix[i][j] = result["solving_time"]
        error_matrix[i][j] = result["values"]["error"]

In [ ]:
from matplotlib.colors import Normalize


Antennas_spaces, Antennas_populations = np.meshgrid(antennas_spaces, antennas_populations)

norms = [
    Normalize(vmin=0, vmax=60),
    Normalize(vmin=0, vmax=5),
]

fig,ax = plt.subplots(figsize=(8,6))
mappable = ax.pcolormesh(Antennas_populations, Antennas_spaces, solve_time_matrix, cmap='plasma', norm=norms[0])
ax.set_xlabel("Antennas Population")
ax.set_ylabel("Antennas Space[m]")
cb=fig.colorbar(mappable, ax=ax)
cb.set_label("Solve time[s]")

fig,ax = plt.subplots(figsize=(8,6))
mappable = ax.pcolormesh(Antennas_populations, Antennas_spaces, error_matrix, cmap='plasma', norm=norms[0])
ax.set_xlabel("Antennas Population")
ax.set_ylabel("Antennas Space[m]")
cb=fig.colorbar(mappable, ax=ax)
cb.set_label("Error[m]")